In [1]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm

def last_changed():
    query = f"""
                SELECT entity_id, max(last_changed) as last_changed, COUNT(*) as nunique
                FROM states
                GROUP BY entity_id
              """
    return (
        pd.read_sql_query(query, 'sqlite:////config/home-assistant_v2.db')
        .assign(last_changed=lambda x: pd.to_datetime(x['last_changed']))
        .set_index('last_changed')
        .sort_index()
    )

def change_entity_id(old, new):
    query = f"""
                UPDATE
                SET entity_id = '{new}'
                WHERE entity_id = '{old}'
              """

def drop(entity_id):
    import sqlite3
    query = f"""
                DELETE
                FROM states
                WHERE entity_id = '{entity_id}'
              """
    con = sqlite3.connect('////config/home-assistant_v2.db')
    cur = con.cursor()
    cur.execute(query)
    con.commit()
    con.close()

# Fetch entities

In [2]:
%%time

def fetch_entities():
    query = F"""SELECT DISTINCT(entity_id) FROM states"""
    return (
        pd.read_sql_query(query, 'sqlite:////config/home-assistant_v2.db')
        .assign(device_class = lambda x: x['entity_id'].str.split('.').str[0])
    )

entities = fetch_entities()

CPU times: user 434 ms, sys: 41 ms, total: 475 ms
Wall time: 511 ms


In [17]:
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(entities)

,entity_id,device_class
0,automation.attic_motion,automation
1,automation.attic_remote,automation
2,automation.back_up_log,automation
3,automation.bathroom,automation
4,automation.bedroom,automation
5,automation.bedroom_control,automation
6,automation.bedroom_notification,automation
7,automation.bedroom_switch,automation
8,automation.bike,automation
9,automation.bike_button,automation


# Remove entities

In [12]:
def drop(entity_id):
    import sqlite3
    query = f"""
                DELETE
                FROM states
                WHERE entity_id = '{entity_id}'
              """
    con = sqlite3.connect('////config/home-assistant_v2.db')
    cur = con.cursor()
    cur.execute(query)
    con.commit()
    con.close()

to_be_removed = [entity for entity in entities['entity_id'] if 'dcf3a3103x' in entity]

In [13]:
to_be_removed

['sensor.dcf3a3103x_all_batteries_charged_today',
 'sensor.dcf3a3103x_all_batteries_discharged_today',
 'sensor.dcf3a3103x_battery_1_charging_w',
 'sensor.dcf3a3103x_battery_1_discharging_w',
 'sensor.dcf3a3103x_battery_2_charging_w',
 'sensor.dcf3a3103x_battery_2_discharging_w',
 'sensor.dcf3a3103x_energy_today_input_3',
 'sensor.dcf3a3103x_energy_today_input_4',
 'sensor.dcf3a3103x_export_to_grid_today',
 'sensor.dcf3a3103x_input_3_amperage',
 'sensor.dcf3a3103x_input_3_voltage',
 'sensor.dcf3a3103x_input_3_wattage',
 'sensor.dcf3a3103x_input_4_amperage',
 'sensor.dcf3a3103x_input_4_voltage',
 'sensor.dcf3a3103x_input_4_wattage',
 'sensor.dcf3a3103x_lifetime_total_all_batteries_charged',
 'sensor.dcf3a3103x_lifetime_total_all_batteries_discharged',
 'sensor.dcf3a3103x_lifetime_total_battery_1_charged',
 'sensor.dcf3a3103x_lifetime_total_battery_1_discharged',
 'sensor.dcf3a3103x_lifetime_total_battery_2_charged',
 'sensor.dcf3a3103x_lifetime_total_battery_2_discharged',
 'sensor.dcf3

In [14]:
for item in to_be_removed:
    print(item)
    drop(item)

sensor.dcf3a3103x_all_batteries_charged_today
sensor.dcf3a3103x_all_batteries_discharged_today
sensor.dcf3a3103x_battery_1_charging_w
sensor.dcf3a3103x_battery_1_discharging_w
sensor.dcf3a3103x_battery_2_charging_w
sensor.dcf3a3103x_battery_2_discharging_w
sensor.dcf3a3103x_energy_today_input_3
sensor.dcf3a3103x_energy_today_input_4
sensor.dcf3a3103x_export_to_grid_today
sensor.dcf3a3103x_input_3_amperage
sensor.dcf3a3103x_input_3_voltage
sensor.dcf3a3103x_input_3_wattage
sensor.dcf3a3103x_input_4_amperage
sensor.dcf3a3103x_input_4_voltage
sensor.dcf3a3103x_input_4_wattage
sensor.dcf3a3103x_lifetime_total_all_batteries_charged
sensor.dcf3a3103x_lifetime_total_all_batteries_discharged
sensor.dcf3a3103x_lifetime_total_battery_1_charged
sensor.dcf3a3103x_lifetime_total_battery_1_discharged
sensor.dcf3a3103x_lifetime_total_battery_2_charged
sensor.dcf3a3103x_lifetime_total_battery_2_discharged
sensor.dcf3a3103x_lifetime_total_energy_input_3
sensor.dcf3a3103x_lifetime_total_energy_input_4
s

In [16]:
def change_entity_id(old, new):
    query = f"""
                UPDATE
                SET entity_id = '{new}'
                WHERE entity_id = '{old}'
              """

replace = {'dcf3a3103x': 'growatt'}
for entity in tqdm(entities['entity_id']):
    if 'dcf3a3103x' in entity:
        old = entity
        new = entity.replace('dcf3a3103x', 'growatt')
        change_entity_id(old, new)

  0%|          | 0/583 [00:00<?, ?it/s]

In [128]:
%%time
df = last_changed()

CPU times: user 35 s, sys: 40.5 s, total: 1min 15s
Wall time: 2min 53s


In [129]:
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 10):
    display(df)

,entity_id,nunique
last_changed,,
2021-08-07 20:18:52.596897,climate.attic,7968
2021-08-07 20:18:52.597377,climate.bathroom,7296
2021-08-07 20:18:52.597669,climate.bedroom,5261
2021-08-07 21:14:07.181818,device_tracker.samsung,65
2021-08-07 21:14:07.183081,sensor.samsung_batterijniveau,347
2021-08-07 21:14:07.183719,sensor.samsung_batterij_laadstatus,71
2021-08-07 21:14:07.184329,binary_sensor.samsung_is_aan_het_opladen,71
2021-08-07 21:14:07.184895,sensor.samsung_type_oplader,71
2021-08-07 21:14:07.185477,sensor.samsung_batterijstatus,65
